Importing Models

In [2]:
!pip install --upgrade transformers==4.38.0 accelerate datasets evaluate rouge-score sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 110.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c1e88b026c15310aa4b0ff7a17df01e807fa9150b1a7d6241c3e4464fe85e6ea
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: toke

In [3]:
!pip install --upgrade peft
!pip install --upgrade transformers accelerate datasets evaluate rouge-score sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0
    Uninstalling transformers-4.38.0:
      Successfully uninstalled transformers-4.38.0


load the data

In [6]:
import pandas as pd
from datasets import Dataset

import pandas as pd

df = pd.read_csv(
    "/content/Reviews.csv",
    on_bad_lines="skip",
    engine="python"
)

Remove short text and summaries

In [7]:
df = df[['Text', 'Summary']].dropna().rename(columns={'Text': 'text', 'Summary': 'summary'})

df = df[df['text'].str.len() > 30]
df = df[df['summary'].str.len() > 5]


 Convert to Hugging Face Dataset and split

In [8]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

Tokenization

In [9]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/496088 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/55121 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import evaluate

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 4) for k, v in result.items()}


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=1,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch"

)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-706713377.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation on Rouge

In [13]:
import torch
from tqdm import tqdm

rouge = evaluate.load("rouge")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_texts = dataset["test"]["text"][:50]
reference_summaries = dataset["test"]["summary"][:50]

generated_summaries = []

print("Generating summaries...")
for text in tqdm(test_texts):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
       summary_ids = model.generate(
    **inputs,
    max_length=128,
    min_length=20,
    num_beams=2,
    do_sample=True,
    temperature=0.8,
    repetition_penalty=2.0,   # discourages repeating words/phrases
    length_penalty=1.2,       # balances concise vs. long summaries
    early_stopping=True
)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print("\nROUGE Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value * 100:.2f}")


Generating summaries...


100%|██████████| 50/50 [04:41<00:00,  5.64s/it]



ROUGE Evaluation Results:
rouge1: 9.95
rouge2: 2.55
rougeL: 8.55
rougeLsum: 8.51


Evaluation on BertScore

In [14]:
!pip install bert_score
bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang="en")
print(results["f1"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[0.8674198389053345, 0.8293523192405701, 0.8513654470443726, 0.8467472791671753, 0.8179668188095093, 0.8286201357841492, 0.856872022151947, 0.8460502624511719, 0.8605809807777405, 0.8149046897888184, 0.8586218357086182, 0.8770459890365601, 0.8471300005912781, 0.8135532140731812, 0.885547935962677, 0.8870041370391846, 0.8298844695091248, 0.8481336832046509, 0.8510894775390625, 0.8183675408363342, 0.848672091960907, 0.8244554996490479, 0.8597766757011414, 0.802621066570282, 0.8120017647743225, 0.8518077731132507, 0.8003877401351929, 0.8335590362548828, 0.8207235932350159, 0.8325726985931396, 0.7965593338012695, 0.8269228339195251, 0.8185100555419922, 0.8474128842353821, 0.785800039768219, 0.8522135615348816, 0.8276985883712769, 0.8791556358337402, 0.8280751705169678, 0.7868818640708923, 0.8549809455871582, 0.8202635049819946, 0.8562108278274536, 0.8325287103652954, 0.8355238437652588, 0.8703986406326294, 0.8291007876396179, 0.8557071685791016, 0.8495057225227356, 0.8351539373397827]


Inference

In [15]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_corpus = [
    """I ordered the grilled chicken sandwich, and it turned out better than I expected.
     The bread was toasted perfectly and had a nice crunch. The chicken was juicy and seasoned with
     herbs that added a great flavor. The fries on the side were crispy and not too oily.
     I would definitely recommend this meal to anyone who loves simple yet delicious food."""
]


inputs = tokenizer(["summarize: " + doc for doc in test_corpus], return_tensors="pt", max_length=512, truncation=True)

inputs = {k: v.to(device) for k, v in inputs.items()}

# generate summary
with torch.no_grad():
   summary_ids = model.generate(
    **inputs,
    max_length=80,
    min_length=20,
    num_beams=4,
    do_sample=True,
    temperature=0.7,
    repetition_penalty=2.0,
    length_penalty=1.0,
    early_stopping=True
)

print("Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))

Summary: Great grilled chicken sandwich! The bread was toasted perfectly and had a nice crunch. The chicken was juicy and seasoned with herbs that added a great flavor. The fries on the side were crispy and not oily.


Save the Model

In [18]:
trainer.save_model("./t5-summarizer_results")
tokenizer.save_pretrained("./t5-summarizer_results")

('./t5-summarizer_results/tokenizer_config.json',
 './t5-summarizer_results/special_tokens_map.json',
 './t5-summarizer_results/spiece.model',
 './t5-summarizer_results/added_tokens.json',
 './t5-summarizer_results/tokenizer.json')